# MODELCHAIN

In [72]:
import pandas as pd
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

In [73]:
temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

In [74]:
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

cec_module = cec_modules['Canadian_Solar_Inc__CS6X_320P']
cec_inverter = cec_inverters['INGETEAM_POWER_TECHNOLOGY_S_A___Ingecon_Sun_40TL_U_M__480V_']

In [75]:
location = Location(latitude= 20.56, longitude= -103.22, altitude=1544, tz='America/Mexico_City')

In [76]:
surface_tilt = 25
surface_azimuth = 180 # pvlib uses 0=North, 90=East, 180=South, 270=West convention
albedo = 0.2

In [77]:
system = PVSystem(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth,
                 module_parameters=cec_module,
                 inverter_parameters=cec_inverter,
                 temperature_model_parameters=temperature_model_parameters,
                 albedo=albedo, surface_type=None, module=None, 
                 module_type='glass_polymer',
                 modules_per_string=19.5, 
                 strings_per_inverter=8, inverter=None, 
                 racking_model='open_rack', 
                 losses_parameters=None, 
                 name='Name of PV system: Huerto de Cutonalá')

In [78]:
#system

In [95]:
mc = ModelChain(system, location, name='Huerto de Cutonalá', 
                clearsky_model='ineichen', transposition_model='haydavies', 
                solar_position_method='nrel_numpy', airmass_model='kastenyoung1989',
                dc_model=None, #from module specifications
                ac_model='sandia', #from inverter specifications
                aoi_model='no_loss', 
                spectral_model='no_loss', #spectral model 'no loss' otherwise error
                temperature_model='sapm', losses_model='no_loss')

In [96]:
#mc

In [97]:
import datetime
#Variables de tiempo

start = pd.Timestamp(datetime.date.today(), tz='America/Mexico_City') # today's date
end = start + pd.Timedelta(days=7) # 7 days from today

In [98]:
from pvlib import solarposition, irradiance, atmosphere, pvsystem, inverter, temperature, iam
from pvlib.forecast import GFS, NAM, NDFD, RAP, HRRR

In [99]:
# Se define el modelo de pronostico
fm = GFS()
#fm = NAM()
#fm = NDFD()
#fm = RAP()
#fm = HRRR()

In [100]:
# Se obtiene la data y se procesa del modelo de pronostico
forecast_data = fm.get_processed_data(20.56, -103.22, start, end)

In [101]:
#pd.set_option('display.max_columns', None)
#forecast_data.head()

In [102]:
ghi = forecast_data['ghi']
dni = forecast_data['dni']
dhi = forecast_data['dhi']
temp_air = forecast_data['temp_air']
wind_speed = forecast_data['wind_speed']
indexDF = forecast_data.index

In [103]:
weather = pd.DataFrame([dni, ghi, dhi, temp_air, wind_speed]).T

In [104]:
mc.run_model(weather)

ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

In [105]:
mc.ac

2020-12-08 06:00:00-06:00      -11.985000
2020-12-08 09:00:00-06:00     5087.140617
2020-12-08 12:00:00-06:00    12801.412488
2020-12-08 15:00:00-06:00    12624.534931
2020-12-08 18:00:00-06:00        9.088758
2020-12-08 21:00:00-06:00      -11.985000
2020-12-09 00:00:00-06:00      -11.985000
2020-12-09 03:00:00-06:00      -11.985000
2020-12-09 06:00:00-06:00      -11.985000
2020-12-09 09:00:00-06:00     6169.882488
2020-12-09 12:00:00-06:00    14539.744943
2020-12-09 15:00:00-06:00     9602.806658
2020-12-09 18:00:00-06:00       46.935173
2020-12-09 21:00:00-06:00      -11.985000
2020-12-10 00:00:00-06:00      -11.985000
2020-12-10 03:00:00-06:00      -11.985000
2020-12-10 06:00:00-06:00      -11.985000
2020-12-10 09:00:00-06:00    21400.027232
2020-12-10 12:00:00-06:00    39950.000000
2020-12-10 15:00:00-06:00    35003.822382
2020-12-10 18:00:00-06:00      133.558563
2020-12-10 21:00:00-06:00      -11.985000
2020-12-11 00:00:00-06:00      -11.985000
2020-12-11 03:00:00-06:00      -11

In [106]:
mc.effective_irradiance

2020-12-08 06:00:00-06:00      0.000000
2020-12-08 09:00:00-06:00    103.191111
2020-12-08 12:00:00-06:00    255.691006
2020-12-08 15:00:00-06:00    259.133998
2020-12-08 18:00:00-06:00      2.137763
2020-12-08 21:00:00-06:00      0.000000
2020-12-09 00:00:00-06:00      0.000000
2020-12-09 03:00:00-06:00      0.000000
2020-12-09 06:00:00-06:00      0.000000
2020-12-09 09:00:00-06:00    125.122525
2020-12-09 12:00:00-06:00    295.518258
2020-12-09 15:00:00-06:00    196.921748
2020-12-09 18:00:00-06:00      3.096599
2020-12-09 21:00:00-06:00      0.000000
2020-12-10 00:00:00-06:00      0.000000
2020-12-10 03:00:00-06:00      0.000000
2020-12-10 06:00:00-06:00      0.000000
2020-12-10 09:00:00-06:00    436.777609
2020-12-10 12:00:00-06:00    945.091418
2020-12-10 15:00:00-06:00    777.650772
2020-12-10 18:00:00-06:00      5.158126
2020-12-10 21:00:00-06:00      0.000000
2020-12-11 00:00:00-06:00      0.000000
2020-12-11 03:00:00-06:00      0.000000
2020-12-11 06:00:00-06:00      0.000000


In [107]:
mc.airmass

,airmass_relative,airmass_absolute
2020-12-08 06:00:00-06:00,NaN,NaN
2020-12-08 09:00:00-06:00,2.886681,2.395973
2020-12-08 12:00:00-06:00,1.406016,1.167007
2020-12-08 15:00:00-06:00,1.715433,1.423826
2020-12-08 18:00:00-06:00,18.932328,15.714013
2020-12-08 21:00:00-06:00,NaN,NaN
2020-12-09 00:00:00-06:00,NaN,NaN
2020-12-09 03:00:00-06:00,NaN,NaN
2020-12-09 06:00:00-06:00,NaN,NaN
2020-12-09 09:00:00-06:00,2.905576,2.411656


In [108]:
mc.cec

<bound method ModelChain.cec of ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses>

In [109]:
mc.cell_temperature

2020-12-08 06:00:00-06:00    13.304749
2020-12-08 09:00:00-06:00    14.849089
2020-12-08 12:00:00-06:00    19.365076
2020-12-08 15:00:00-06:00    25.877347
2020-12-08 18:00:00-06:00    31.285581
2020-12-08 21:00:00-06:00    29.402191
2020-12-09 00:00:00-06:00    18.017914
2020-12-09 03:00:00-06:00    15.238037
2020-12-09 06:00:00-06:00    14.003265
2020-12-09 09:00:00-06:00    16.908342
2020-12-09 12:00:00-06:00    24.315839
2020-12-09 15:00:00-06:00    23.688893
2020-12-09 18:00:00-06:00    32.185004
2020-12-09 21:00:00-06:00    31.409882
2020-12-10 00:00:00-06:00    18.912842
2020-12-10 03:00:00-06:00    15.192535
2020-12-10 06:00:00-06:00    13.458618
2020-12-10 09:00:00-06:00    26.588806
2020-12-10 12:00:00-06:00    43.464736
2020-12-10 15:00:00-06:00    44.744162
2020-12-10 18:00:00-06:00    30.552687
2020-12-10 21:00:00-06:00    29.598419
2020-12-11 00:00:00-06:00    18.973175
2020-12-11 03:00:00-06:00    14.991577
2020-12-11 06:00:00-06:00    13.718842
2020-12-11 09:00:00-06:00

In [110]:
mc.solar_position

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2020-12-08 06:00:00-06:00,107.715367,107.715367,-17.715367,-17.715367,108.327857,7.931504
2020-12-08 09:00:00-06:00,69.879035,69.916706,20.120965,20.083294,125.298173,7.875987
2020-12-08 12:00:00-06:00,44.730395,44.744260,45.269605,45.255740,165.177690,7.820347
2020-12-08 15:00:00-06:00,54.421896,54.440999,35.578104,35.559001,218.974696,7.764585
2020-12-08 18:00:00-06:00,87.903717,88.128916,2.096283,1.871084,244.722970,7.708702
2020-12-08 21:00:00-06:00,127.954555,127.954555,-37.954555,-37.954555,256.503283,7.652699
2020-12-09 00:00:00-06:00,169.236595,169.236595,-79.236595,-79.236595,255.604062,7.596576
2020-12-09 03:00:00-06:00,148.707289,148.707289,-58.707289,-58.707289,100.493823,7.540335
2020-12-09 06:00:00-06:00,107.840226,107.840226,-17.840226,-17.840226,108.396286,7.483978
2020-12-09 09:00:00-06:00,70.017722,70.055489,19.982278,19.944511,125.320525,7.427503


In [111]:
mc.ac_model

<bound method ModelChain.snlinverter of ModelChain: 
  name: Huerto de Cutonalá
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: cec
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses>